The elevation data is from National Land Survey of Iceland, [link here](https://ftp.lmi.is/gisdata/raster/).

It is originally a `geotiff` (*geo* for *georeferenced*) file which we need to clip (I use the open-source `QGis`) and turn into a mesh (using Python) to allow for plotting in `bokeh`.

This notebook makes use of `gdal` which is infernally difficult to install. To avoid the hassle I recommend avoiding running it on a platform like Google Colab which has those packages preinstalled.

In [ ]:
from osgeo import gdal
import pandas as pd
import numpy as np

## Instructions for Clipping Raster Data
Install [QGis](https://qgis.org/en/site/forusers/download.html). Import the extracted `geotiff` file. The workflow is then as follows:

Raster → Extraction → Clip raster by extent → Clipping Extent → Draw on Canvas

In [ ]:
location = 'Svartsengi'

In [ ]:
ds = gdal.Open(f'data/{location}.tif')

array = ds.GetRasterBand(1).ReadAsArray()
flattened = array.flatten()
geotransform = ds.GetGeoTransform()
res = geotransform[1]
x_min = geotransform[0]
y_max = geotransform[3]
x_size = ds.RasterXSize
y_size = ds.RasterYSize
x_start = x_min + res / 2
y_start = y_max - res / 2
ds = None

x = np.arange(x_start, x_start + x_size * res, res)
y = np.arange(y_start, y_start - y_size * res, -res)
x = np.tile(x, y_size)
y = np.repeat(y, x_size)

dictionary = {'x': x,
              'y': y,
              'value': flattened}
df = pd.DataFrame(dictionary)
print(df.head())
df.to_csv(f'data/{location}.csv')